## Deliverable 2. Create a Customer Travel Destinations Map.

#### is the below API key setup legit?

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
g_key=os.environ.get('g_key')

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luderitz,NaN,-26.6481,15.1594,73.90,54,0,9.40,clear sky
1,1,Albany,US,42.6001,-73.9662,21.99,48,100,3.24,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,71.60,68,40,5.75,scattered clouds
3,3,Kapaa,US,22.0752,-159.3190,71.60,73,75,24.16,light rain
4,4,Atuona,PF,-9.8000,-139.0333,79.93,75,7,13.38,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# extra prnt check
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Atuona,PF,-9.8000,-139.0333,79.93,75,7,13.38,clear sky
6,6,Kulhudhuffushi,MV,6.6221,73.0700,83.08,68,78,9.35,broken clouds
11,11,Faanui,PF,-16.4833,-151.7500,83.25,70,96,1.25,light rain
12,12,Rikitea,PF,-23.1203,-134.9692,78.37,78,100,8.52,overcast clouds
13,13,East London,ZA,-33.0153,27.9116,82.40,57,0,3.44,clear sky
20,20,Port Elizabeth,ZA,-33.9180,25.5701,75.20,73,20,17.27,few clouds
25,25,Butaritari,KI,3.0707,172.7902,81.66,83,1,14.18,clear sky
26,26,Souillac,MU,-20.5167,57.5167,86.00,62,40,12.66,light rain
27,27,Avarua,CK,-21.2078,-159.7750,75.20,100,100,2.30,overcast clouds
29,29,Bubaque,GW,11.2833,-15.8333,78.84,67,0,6.11,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                162
City                   162
Country                162
Lat                    162
Lng                    162
Max Temp               162
Humidity               162
Cloudiness             162
Wind Speed             162
Current Description    162
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no rows to drop
clean_df = preferred_cities_df.dropna()

In [11]:
# extra print - check no full rows dropped
clean_df.count()

City_ID                162
City                   162
Country                162
Lat                    162
Lng                    162
Max Temp               162
Humidity               162
Cloudiness             162
Wind Speed             162
Current Description    162
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,79.93,clear sky,-9.8000,-139.0333,
6,Kulhudhuffushi,MV,83.08,broken clouds,6.6221,73.0700,
11,Faanui,PF,83.25,light rain,-16.4833,-151.7500,
12,Rikitea,PF,78.37,overcast clouds,-23.1203,-134.9692,
13,East London,ZA,82.40,clear sky,-33.0153,27.9116,
20,Port Elizabeth,ZA,75.20,few clouds,-33.9180,25.5701,
25,Butaritari,KI,81.66,clear sky,3.0707,172.7902,
26,Souillac,MU,86.00,light rain,-20.5167,57.5167,
27,Avarua,CK,75.20,overcast clouds,-21.2078,-159.7750,
29,Bubaque,GW,78.84,clear sky,11.2833,-15.8333,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# extra print check
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,79.93,clear sky,-9.8000,-139.0333,Villa Enata
6,Kulhudhuffushi,MV,83.08,broken clouds,6.6221,73.0700,Haajy Guest House 1
11,Faanui,PF,83.25,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
12,Rikitea,PF,78.37,overcast clouds,-23.1203,-134.9692,Pension Maro'i
13,East London,ZA,82.40,clear sky,-33.0153,27.9116,Tu Casa
20,Port Elizabeth,ZA,75.20,few clouds,-33.9180,25.5701,39 On Nile Guest House
25,Butaritari,KI,81.66,clear sky,3.0707,172.7902,Isles Sunset Lodge
26,Souillac,MU,86.00,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
27,Avarua,CK,75.20,overcast clouds,-21.2078,-159.7750,Paradise Inn
29,Bubaque,GW,78.84,clear sky,11.2833,-15.8333,"Casa Dora, Bubaque"


In [15]:
hotel_df.count()

City                   162
Country                162
Max Temp               162
Current Description    162
Lat                    162
Lng                    162
Hotel Name             162
dtype: int64

In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [17]:
# extra print check
clean_hotel_df.count()

City                   162
Country                162
Max Temp               162
Current Description    162
Lat                    162
Lng                    162
Hotel Name             162
dtype: int64

In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
# max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))